In [3]:
# !pip install openai

In [4]:
import openai 
import config
import os

In [5]:
os.environ['OPENAI_API_KEY'] = config.API_KEY   

In [6]:
from openai import OpenAI 

client = OpenAI()

# Create Training file

In [23]:
file = client.files.create(
    file=open("../data/training-set-with-sysprompt.jsonl", mode="rb"),
    purpose="fine-tune"    
)
file_id = file.id 
# client.files.create(
#     file=open("../data/training-set-with-sysprompt.jsonl", mode="rb"),
#     purpose="fine-tune"    
# )

In [24]:
file_id

'file-z5cYJLbkVmG3oRLYlsoyChGy'

In [25]:
job_id= client.fine_tuning.jobs.create(
  training_file=file_id,
  model="gpt-3.5-turbo",
).id


In [26]:
print(job_id)

ftjob-aVrRoNWdyQfNOfw82f0BNTwF


# Fine-tune model on the training file

In [30]:
client.fine_tuning.jobs.list(limit=10).data

FineTuningJob(id='ftjob-zDvzuLghejvvMK8KPitcTPWI', created_at=1712640715, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model='ft:gpt-3.5-turbo-0125:hasten-inc::9BymRBoU', finished_at=1712642274, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=8), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-M80H3eXQPwPG2R3uDgRlmsWo', result_files=['file-9gjXwOdDpwOYZmfFknPtcuFh'], status='succeeded', trained_tokens=1130697, training_file='file-m32UGveurP8fEPZUeQQ92NDU', validation_file=None, user_provided_suffix=None, seed=992399036, integrations=[])

In [31]:
# # List 10 fine-tuning jobs
job_num = -1 # choosing oldest model; note: latest created will be assigned index 0; FIFO
item = client.fine_tuning.jobs.list(limit=10).data[job_num]
job_id = item.id
training_file = item.training_file
print(job_id)
print(training_file)

ftjob-zDvzuLghejvvMK8KPitcTPWI
file-m32UGveurP8fEPZUeQQ92NDU


In [37]:
fine_tuned_model_name = client.fine_tuning.jobs.retrieve(job_id).fine_tuned_model
# print(fine_tuned_model_name)

# Use the fine-tuned model

In [19]:
file = open("../data/system_prompt_enhanced.txt")

In [20]:
system_prompt = file.read() # to modify as desired
prompt = "spawn 6 trees in a grid"

In [35]:
completion = client.chat.completions.create(
  model = fine_tuned_model_name,
  messages=[
    {"role": "system", "content": f"{system_prompt}"},
    {"role": "user", "content": f"{prompt}"}
  ]
)

In [36]:
print(completion.choices[0].message.content)


   @f3( x 0.0 z)
   = position

   position
   Spawn(Name: tree  Quantity: 6  Gap: 2.0)
  
   @f3( x 0.0 z)
   = position

   Get(position)
   Spawn(Name: tree  Quantity: 6  Gap: 2.0)
  
    This Shard will spawn a number of Entities with the specified unique Name, in the 3D space. The spawned entities will be spawned in a grid like pattern with 
    a specified Gap between each entity. The spawned entities will have their 0,0,0 point snapped to the position provided in the solution above, tree 1 will be spawned 2 units
    to the right of the origin, followed by tree 2, 2 units to the right of tree 1 and so forth. If the spawned quantity exceeds  the amount of entities that can  fit in
    the grid (7 in the solution provided above) the Shard will only spawn the amount of entities that can fit in the grid. 
    #General
